<a href="https://colab.research.google.com/github/aishoo1612/Computer-Vision/blob/master/Transfer_Learning_(Classification_of_Ants_and_Bees)With_RESNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import torch 
import torch.nn as nn 
import torchvision 
from torchvision import transforms, datasets, models
import torch.optim as optim 
import matplotlib.pyplot as plt 



In [3]:
def set_para_requires_grad( model, extracting):
  if extracting:
    for param in model.parameters():
      param.requires_grad = False 
      


In [5]:
def train_model(model, data_loaders, criterion, optimizer, epochs=25 ):
  for epoch in range(epochs):
    print('Epochs-%d', epoch)
    print('-'*15)

    for phase in ['train', 'val']:
      if phase == 'train':
        model.train()
      else :
        model.eval()

    running_loss = 0.0
    correct_guess = 0


    for inputs, labels in data_loader[phase]:
      inputs = inputs.to(device)
      labels = labels.to(device)


      optimizer.zero_grad()

      with torch.set_grad_enabled(phase == 'train'):
        outputs = model(inputs)
        loss = criterion(outputs, labels)


        _, preds = torch.max(outputs, 1)

        if phase == 'train':
          loss.backward()
          optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        correct_guess += torch.sum(preds == labels.data)

      epoch_loss = running_loss/len(data_loaders[phase].dataset)
      epoch_acc = correct_guess.double()/len(data_loaders[phase].dataset)

      print('{} Loss: {.4f}, {} Acc : {.4f}'.format(phase, epoch_loss, epoch_acc))

root_dir = 'hymenopetra_data/'


image_transforms ={
      'train' : transforms.Compose([transforms.RandomRotation(-270, 270),
                transforms.Resize(224,224)
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.485, 0.456, 0.406],std = [0.229, 0.225, 0.224])]),
      'val' :   transforms.Compose([transforms.RandomRotation(-270, 270),
                transforms.Resize(224,224)
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.485, 0.456, 0.406],std = [0.229, 0.225, 0.224])]),

  }

data_generator = {k:datasets.ImageFolder(os.path.join(root_dir, k), image_transforms[k] for k in ['train','val'])}

data_loader = {k:torch.utils.data.DataLoader(data_generator[k], batch_size =2, shuffle= True, num_workers=4) for k in ['train', 'val']}

device = torch.device('cuda : 0' if torch.cuda.is.available() else 'cpu')

model = models.resnet18(pretrained=True)

set_para_requires_grad(model, True)

num_features = model.fc.in_features
model.fc = nn.Linear(features, 2)

model.to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters, lr =0.001)

params_to_update = []

for name, param in model.named_parameters:
  if param.requires_grad == True:
    params_to_update.append(param)
    print("\t",name)



train_model(model, data_loader, criterion, optimizer)











SyntaxError: ignored